# Levy Function Optimization

## 1. CUDA Parallel Implementation

In [3]:

%%writefile levy_cuda.cu

#include <iostream>
#include <cmath>
#include <curand_kernel.h>
#include <cfloat>
#include <chrono>
#include <fstream>  // Required for CSV file I/O
#include <iomanip>  // Required for precision

using namespace std;

// ------------------------------
// PARAMETERS
// ------------------------------
#define POP 256
#define DIM 5
#define MAX_IT 8000
#define LB -100.0
#define UB 100.0

#define BLOCK_SIZE 256

// ------------------------------
// DEVICE HELPER FUNCTIONS
// ------------------------------
// Generate random double [0, 1]
__device__ double randDouble(curandState* state) {
    return curand_uniform_double(state);
}

// Generate random integer [a, b]
__device__ int randInt(curandState* state, int a, int b) {
    float r = curand_uniform(state);
    return (int)(a + r * (b - a + 0.99999f));
}

// ------------------------------
// FITNESS FUNCTION: LEVY
// ------------------------------
__device__ double fitness(double* x, int dim) {
    const double PI = 3.14159265358979323846;
    double w0 = 1.0 + (x[0] - 1.0) / 4.0;
    double wd = 1.0 + (x[dim-1] - 1.0) / 4.0;

    double term1 = pow(sin(PI * w0), 2);
    double term3 = pow(wd - 1.0, 2) * (1.0 + pow(sin(2.0 * PI * wd), 2));

    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double wi = 1.0 + (x[i] - 1.0) / 4.0;
        sum += pow(wi - 1.0, 2) * (1.0 + 10.0 * pow(sin(PI * wi + 1.0), 2));
    }
    return term1 + sum + term3;
}

// ------------------------------
// KERNELS
// ------------------------------
__global__ void init_population_kernel(double* pop, curandState* states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curand_init(seed, idx, 0, &states[idx]);
        for (int d = 0; d < DIM; d++) {
            pop[idx * DIM + d] = LB + randDouble(&states[idx]) * (UB - LB);
        }
    }
}

__global__ void evaluate_fitness_kernel(double* pop, double* fitness_vals) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        double sol[DIM];
        for (int d = 0; d < DIM; d++) {
            sol[d] = pop[idx * DIM + d];
        }
        fitness_vals[idx] = fitness(sol, DIM);
    }
}

__global__ void loa_update_kernel(double* pop, double* fitness_vals, double* new_pop, double* new_fitness_vals, curandState* states, int t) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curandState localState = states[idx];
        double myFit = fitness_vals[idx];

        // 1. Find better candidates
        int betterCount = 0;
        int betterIdx = -1;

        for (int j = 0; j < POP; j++) {
            if (fitness_vals[j] < myFit) {
                betterCount++;
            }
        }

        if (betterCount > 0) {
            int pick = randInt(&localState, 0, betterCount - 1);
            int current = 0;
            for (int j = 0; j < POP; j++) {
                if (fitness_vals[j] < myFit) {
                    if (current == pick) {
                        betterIdx = j;
                        break;
                    }
                    current++;
                }
            }
        }

        // 2. Generate Candidate
        double candidate[DIM];
        double currentPos[DIM];
        for(int d=0; d<DIM; d++) currentPos[d] = pop[idx * DIM + d];

        double rp = randDouble(&localState);

        if (rp < 0.5 && betterIdx != -1) {
            // Escape Move
            double betterPos[DIM];
            for(int d=0; d<DIM; d++) betterPos[d] = pop[betterIdx * DIM + d];

            for (int d = 0; d < DIM; d++) {
                double r = randDouble(&localState);
                int I = randInt(&localState, 1, 2);
                candidate[d] = currentPos[d] + r * (betterPos[d] - I * currentPos[d]);

                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        } else {
            // Hide Move
            for (int d = 0; d < DIM; d++) {
                double r = randDouble(&localState);
                candidate[d] = currentPos[d] + (1.0 - 2.0 * r) * (UB - LB) / (double)t;

                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        }

        // 3. Evaluate and Update
        double f = fitness(candidate, DIM);

        if (f < myFit) {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = candidate[d];
            new_fitness_vals[idx] = f;
        } else {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = currentPos[d];
            new_fitness_vals[idx] = myFit;
        }

        states[idx] = localState;
    }
}

// ------------------------------
// MAIN FUNCTION
// ------------------------------
int main() {
    auto t_start = chrono::high_resolution_clock::now();

    // 1. Initialize CSV File
    ofstream csvFile("levy_data.csv");
    csvFile << "Iteration,Best_Fitness";
    for(int d=0; d<DIM; d++) csvFile << ",x" << d+1;
    csvFile << "\n";
    csvFile << fixed << setprecision(10);

    double *d_pop, *d_fitness, *d_new_pop, *d_new_fitness;
    curandState *d_states;

    // Host buffers for logging
    double h_fitness[POP];
    double h_pop[POP * DIM];

    size_t pop_size = POP * DIM * sizeof(double);
    size_t fit_size = POP * sizeof(double);

    cudaMalloc(&d_pop, pop_size);
    cudaMalloc(&d_fitness, fit_size);
    cudaMalloc(&d_new_pop, pop_size);
    cudaMalloc(&d_new_fitness, fit_size);
    cudaMalloc(&d_states, POP * sizeof(curandState));

    int threads = 128;
    int blocks = (POP + threads - 1) / threads;

    init_population_kernel<<<blocks, threads>>>(d_pop, d_states, time(NULL));
    evaluate_fitness_kernel<<<blocks, threads>>>(d_pop, d_fitness);
    cudaDeviceSynchronize();

    double bestGlobalFit = DBL_MAX;

    cout << "Starting LOA Optimization on Levy..." << endl;

    // ------------------------------
    // OPTIMIZATION LOOP
    // ------------------------------
    for (int it = 1; it <= MAX_IT; it++) {
        // A. Run Optimization Step on GPU
        loa_update_kernel<<<blocks, threads>>>(d_pop, d_fitness, d_new_pop, d_new_fitness, d_states, it);
        cudaDeviceSynchronize();

        // B. Swap Pointers
        double *temp_pop = d_pop; d_pop = d_new_pop; d_new_pop = temp_pop;
        double *temp_fit = d_fitness; d_fitness = d_new_fitness; d_new_fitness = temp_fit;

        // C. Data Logging (HAPPENS EVERY ITERATION)
        // Copy current generation data to CPU
        cudaMemcpy(h_fitness, d_fitness, fit_size, cudaMemcpyDeviceToHost);
        cudaMemcpy(h_pop, d_pop, pop_size, cudaMemcpyDeviceToHost);

        // Find the best individual in this specific generation
        int bestIdx = 0;
        for(int i=1; i<POP; i++) {
            if(h_fitness[i] < h_fitness[bestIdx]) {
                bestIdx = i;
            }
        }

        // Track global best for console output
        if(h_fitness[bestIdx] < bestGlobalFit) bestGlobalFit = h_fitness[bestIdx];

        // Write to CSV: Iteration, Fitness, x1, x2, x3, x4, x5
        csvFile << it << "," << h_fitness[bestIdx];
        for(int d=0; d<DIM; d++) {
            csvFile << "," << h_pop[bestIdx * DIM + d];
        }
        csvFile << "\n";

        // D. Console Progress
        if (it % 1000 == 0) {
             cout << "Iter " << it << " | Best = " << bestGlobalFit << endl;
        }
    }

    csvFile.close();

    // ------------------------------
    // FINAL OUTPUT
    // ------------------------------
    cout << "\nOptimization Complete." << endl;
    cout << "Data saved to 'levy_data.csv' (8000 rows)" << endl;

    // Find final best index from last iteration data
    int bestIdx = 0;
    for(int i=1; i<POP; i++) {
        if(h_fitness[i] < h_fitness[bestIdx]) bestIdx = i;
    }

    cout << "Final Best Solution:\n";
    for(int d=0; d<DIM; d++) {
        cout << "x" << d+1 << " = " << h_pop[bestIdx * DIM + d] << endl;
    }
    cout << "Final Best Value = " << bestGlobalFit << endl;

    // Cleanup
    cudaFree(d_pop);
    cudaFree(d_fitness);
    cudaFree(d_new_pop);
    cudaFree(d_new_fitness);
    cudaFree(d_states);

    auto t_end = chrono::high_resolution_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(t_end - t_start).count();
    cout << "Execution Time = " << elapsed << " sec" << endl;

    return 0;
}

Overwriting levy_cuda.cu


In [4]:
!nvcc -arch=sm_75 levy_cuda.cu -o levy_cuda
!./levy_cuda

Starting LOA Optimization on Levy...
Iter 1000 | Best = 7.65192e-05
Iter 2000 | Best = 6.71358e-05
Iter 3000 | Best = 6.38824e-06
Iter 4000 | Best = 5.88426e-06
Iter 5000 | Best = 5.83233e-06
Iter 6000 | Best = 5.83227e-06
Iter 7000 | Best = 3.97944e-07
Iter 8000 | Best = 3.80856e-07

Optimization Complete.
Data saved to 'levy_data.csv' (8000 rows)
Final Best Solution:
x1 = 1.00012
x2 = 0.999823
x3 = 1.00015
x4 = 0.999183
x5 = 0.999811
Final Best Value = 3.80856e-07
Execution Time = 1.27214 sec


In [5]:

%%writefile levy_serial.cpp
#include <bits/stdc++.h>
using namespace std;

/* ---------------------------------------
      Random Generator
---------------------------------------*/
mt19937 rng(time(NULL));

double randF(double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double levy(const vector<double> &x) {
    int dim = x.size();

    const double PI = 3.14159265358979323846;
    double w0 = 1.0 + (x[0] - 1.0) / 4.0;
    double wd = 1.0 + (x[dim-1] - 1.0) / 4.0;

    double term1 = pow(sin(PI * w0), 2);
    double term3 = pow(wd - 1.0, 2) * (1.0 + pow(sin(2.0 * PI * wd), 2));

    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double wi = 1.0 + (x[i] - 1.0) / 4.0;
        sum += pow(wi - 1.0, 2) * (1.0 + 10.0 * pow(sin(PI * wi + 1.0), 2));
    }
    return term1 + sum + term3;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(LB, UB); // Note: Original code used randF(rng,0,1) here but logic was r*(SSA-I*x).
        // Wait, original code: double r = randF(rng,0,1);
        // Let's stick to original logic.
        double r_val = randF(0, 1);
        int I = randInt(1, 2);
        newX[j] = x[j] + r_val * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search)
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(0, 1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population
---------------------------------------*/
vector<vector<double>> init_population(){
    vector<vector<double>> pop(POP, vector<double>(DIM));
    for(int i=0;i<POP;i++)
        for(int d=0;d<DIM;d++)
            pop[i][d] = randF(LB, UB);
    return pop;
}

/* =======================================
             MAIN LOA SERIAL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    vector<vector<double>> pop = init_population();
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=levy(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        for(int i=0;i<POP;i++){

            vector<int> better;
            for(int j=0;j<POP;j++)
                if(fitness[j]<fitness[i]) better.push_back(j);

            int betterIdx=-1;
            if(!better.empty())
                betterIdx = better[randInt(0,(int)better.size()-1)];

            vector<double> candidate;
            if(randF(0,1)<0.5 && betterIdx!=-1)
                candidate = escape(pop[i],pop[betterIdx]);
            else
                candidate = hide(pop[i],it);

            double f = levy(candidate);

            if(f<fitness[i]){
                pop[i]=candidate;
                fitness[i]=f;
            }
            if(f<bestFit){
                bestFit=f;
                bestSol=candidate;
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Levy" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing levy_serial.cpp


In [6]:
!g++ levy_serial.cpp -o levy_serial
!./levy_serial

Iter 1000 | Best = 0.000135647
Iter 2000 | Best = 4.8256e-06
Iter 3000 | Best = 4.79539e-06
Iter 4000 | Best = 4.79523e-06
Iter 5000 | Best = 4.79505e-06
Iter 6000 | Best = 4.79505e-06
Iter 7000 | Best = 4.79505e-06
Iter 8000 | Best = 9.15411e-07

Final Best Solution:
x1 = 0.999558
x2 = 0.998982
x3 = 1.00003
x4 = 0.999416
x5 = 0.99989

Best Levy Value = 9.15411e-07

Execution Time = 16.8502 sec


## 3. OpenMP Parallel Implementation

In [7]:

%%writefile levy_omp.cpp
#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

/* ---------------------------------------
      Thread-safe Random Generator
---------------------------------------*/
double randF(mt19937 &rng, double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(mt19937 &rng, int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double levy(const vector<double> &x) {
    int dim = x.size();

    const double PI = 3.14159265358979323846;
    double w0 = 1.0 + (x[0] - 1.0) / 4.0;
    double wd = 1.0 + (x[dim-1] - 1.0) / 4.0;

    double term1 = pow(sin(PI * w0), 2);
    double term3 = pow(wd - 1.0, 2) * (1.0 + pow(sin(2.0 * PI * wd), 2));

    double sum = 0.0;
    for (int i = 0; i < dim - 1; ++i) {
        double wi = 1.0 + (x[i] - 1.0) / 4.0;
        sum += pow(wi - 1.0, 2) * (1.0 + 10.0 * pow(sin(PI * wi + 1.0), 2));
    }
    return term1 + sum + term3;

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)  — parallel safe
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA,
                      mt19937 &rng)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        int I = randInt(rng,1,2);
        newX[j] = x[j] + r * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search) — parallel safe
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t,mt19937 &rng){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population (PARALLEL)
---------------------------------------*/
vector<vector<double>> init_population(vector<mt19937> &rngs){
    vector<vector<double>> pop(POP, vector<double>(DIM));

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        mt19937 &local_rng = rngs[tid];

        #pragma omp for schedule(static)
        for(int i=0;i<POP;i++)
            for(int d=0;d<DIM;d++)
                pop[i][d] = randF(local_rng,LB,UB);
    }
    return pop;
}

/* =======================================
             MAIN LOA PARALLEL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    int threads = omp_get_max_threads();
    vector<mt19937> rngs(threads);

    random_device rd;
    for(int i=0;i<threads;i++)
        rngs[i].seed(rd()+i*111);

    vector<vector<double>> pop = init_population(rngs);
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=levy(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
       Full population parallel every step
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        #pragma omp parallel
        {
            int tid = omp_get_thread_num();
            mt19937 &localRng = rngs[tid];

            double localBest = 1e18;
            vector<double> localBestSol(DIM);

            #pragma omp for schedule(static)
            for(int i=0;i<POP;i++){

                vector<int> better;
                for(int j=0;j<POP;j++)
                    if(fitness[j]<fitness[i]) better.push_back(j);

                int betterIdx=-1;
                if(!better.empty())
                    betterIdx = better[randInt(localRng,0,(int)better.size()-1)];

                vector<double> candidate;
                if(randF(localRng,0,1)<0.5 && betterIdx!=-1)
                    candidate = escape(pop[i],pop[betterIdx],localRng);
                else
                    candidate = hide(pop[i],it,localRng);

                double f = levy(candidate);

                if(f<fitness[i]){
                    pop[i]=candidate;
                    fitness[i]=f;
                }
                if(f<localBest){
                    localBest=f;
                    localBestSol=candidate;
                }
            }

            // Update global best safely
            #pragma omp critical
            {
                if(localBest<bestFit){
                    bestFit=localBest;
                    bestSol=localBestSol;
                }
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Levy" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing levy_omp.cpp


In [ ]:
!g++ -fopenmp levy_omp.cpp -o levy_omp
!./levy_omp

Iter 1000 | Best = 0.00019277
Iter 2000 | Best = 3.35095e-05
Iter 3000 | Best = 3.19703e-05
Iter 4000 | Best = 6.11208e-06
Iter 5000 | Best = 5.56797e-06
Iter 6000 | Best = 5.5483e-06
Iter 7000 | Best = 7.57707e-07
Iter 8000 | Best = 3.18375e-07

Final Best Solution:
x1 = 1.00016
x2 = 1.00003
x3 = 0.999737
x4 = 0.999301
x5 = 0.999615

Best Levy Value = 3.18375e-07

Execution Time = 13.2389 sec
